In [ ]:
!pip install requests gradio pillow

In [1]:
with open(".gitignore", "w") as f:
    f.write(".env\n")
    f.write("*.pyc\n")
    f.write("__pycache__/\n")

In [ ]:
import requests
from PIL import Image
import io 
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("HF_API_KEY")
API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-base"
headers = {"Authorization": f"Bearer {API_KEY}"}

def query_blip(image_path):
    with open(image_path, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    result = response.json()
    return result

sample_image = r"C:\Users\aryan\OneDrive\Pictures\AnimeWallpaper\1304790 (1) (1) (1) (1) (1).png"
print(query_blip(sample_image))

In [ ]:
from dotenv import load_dotenv
import os
import requests
import gradio as gr
import io
from PIL import Image
import numpy as np

load_dotenv()
API_KEY = os.getenv("HF_API_KEY")
API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-base"
headers = {"Authorization": f"Bearer {API_KEY}"}

def generate_caption(image):
    if isinstance(image, str):
        image = Image.open(image)
    elif isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    image = image.convert("RGB")
    buf = io.BytesIO()
    image.save(buf, format="JPEG")
    data = buf.getvalue()
    response = requests.post(API_URL, headers=headers, data=data)
    result = response.json()
    if isinstance(result, dict) and "error" in result:
        return "Error: " + result["error"]
    return result[0]["generated_text"]

iface = gr.Interface(
    fn=generate_caption,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Caption Generator",
    description="Upload an image to generate a caption using the BLIP API."
)

iface.launch(share=True)